In [ ]:
import pandas
import seaborn as sns

sns.set_style("whitegrid")
sns.set_context("poster", font_scale=0.75)
sns.set_palette("colorblind")

alloc_order = ["Table", "ArrayAligned", "ArrayLocked", "ArrayAtomic", "ListLocal", "ListLocked", "Kernel"]

# Local Benchmarks

## 4K Pages

In [ ]:
pgd = pandas.concat([
    pandas.read_csv("out/bulk.csv"),
    pandas.read_csv("keep/kernel_bulk.csv"),
])
# print("pages:", pgd["pages"][0])
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="time")
g = sns.relplot(data=pgd, kind="line", x="x", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[1, 2, 4, 6, 8, 10])
g.set(ylim=(0, 450))
g.set(ylabel="time in ns")
g.set(xlabel="cores")
g.set_titles("{col_name}")
g.legend.set_title("Allocator")


In [ ]:
pgd = pandas.concat([
    pandas.read_csv("out/bulk.csv"),
    pandas.read_csv("keep/kernel_bulk.csv"),
])
pgd["get_avg"] = 10**9 / pgd["get_avg"]
pgd["put_avg"] = 10**9 / pgd["put_avg"]
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="ops")
g = sns.relplot(data=pgd, kind="line", x="x", y="ops",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[1, 2, 4, 6, 8, 10])
g.set(ylabel="IO operations / thread / s")
g.set(xlabel="cores")
g.set_titles("{col_name}")
g.legend.set_title("Allocator")


## 2M Pages

In [ ]:
pgd = pandas.read_csv("old/bulk_s1.csv")
print("allocs:", pgd["allocs"][0])
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="time")
g = sns.relplot(data=pgd, kind="line", x="x", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[1, 2, 4, 6, 8, 10])
g.set(ylim=(0, 100))
g.set(ylabel="time in ns")
g.set(xlabel="cores")
g.set_titles("{col_name}")


# Server Benchmarks

In [ ]:
vd = pandas.concat([
    pandas.read_csv("remote/bulk_v.csv"),
    pandas.read_csv("keep/remote/kernel_bulk.csv")
])
vd["mem"] = "DRAM"

pd = pandas.read_csv("remote/bulk_p.csv")
pd["mem"] = "NVRAM"

pgd = pandas.concat([vd, pd], ignore_index=True)
pgd["cores"] = pgd["x"]

pgd["get"] = pgd["get_avg"]
pgd["put"] = pgd["put_avg"]
pgd = pgd[["cores", "alloc", "get", "put", "mem"]].melt(
    id_vars=["cores", "alloc", "mem"],
    value_vars=["get", "put"], value_name="time")
# pgd = pgd.groupby(["cores", "alloc", "variable", "mem"]).min()
g = sns.relplot(data=pgd, kind="line", x="cores", y="time",
                row="variable", col="mem", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[1, 16, 32, 48, 64, 80, 96])
g.set(ylim=(0, 1400))
g.legend.set_title("Allocator")
g.set(ylabel="time in ns")
g.set(xlabel="cores")
g.set_titles("{row_name}, {col_name}")
g.savefig("../../text/thesis/plot/bulk.pdf")


In [ ]:
pgd = pandas.concat([
    pandas.read_csv("remote/bulk_v.csv"),
    pandas.read_csv("keep/remote/kernel_bulk.csv")
])
pgd["get"] = pgd["get_avg"]
pgd["put"] = pgd["put_avg"]
pgd = pgd[["x", "alloc", "get", "put"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get", "put"], value_name="time")
pgd["cores"] = pgd["x"]
# pgd = pgd.groupby(["cores", "alloc", "variable"]).min()
g = sns.relplot(data=pgd, kind="line", x="cores", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[1, 4, 8, 12, 16])
g.set(ylabel="time in ns")
g.set(xlabel="cores")
g.set(ylim=(0, 300))
g.set(xlim=(0, 18))
g.set_titles("{col_name}")
g.legend.set_title("Allocator")
g.savefig("../../text/thesis/plot/bulk-zoom.pdf")


In [ ]:
vd = pandas.concat([
    pandas.read_csv("remote/bulk_v.csv"),
    pandas.read_csv("keep/remote/kernel_bulk.csv")
])
vd["mem"] = "DRAM"

pd = pandas.read_csv("remote/bulk_p.csv")
pd["mem"] = "NVRAM"

pgd = pandas.concat([vd, pd], ignore_index=True)
pgd["cores"] = pgd["x"]
pgd["get"] = pgd["get_avg"]
pgd["put"] = pgd["put_avg"]
pgd = pgd[["cores", "alloc", "get", "put", "mem"]].melt(
    id_vars=["cores", "alloc", "mem"],
    value_vars=["get", "put"], value_name="time")
# pgd = pgd.groupby(["cores", "alloc", "variable", "mem"]).min()
order = ["Table", "ArrayAtomic", "ListLocal", "Kernel"]
g = sns.relplot(data=pgd, kind="line", x="cores", y="time",
                row="variable", col="mem", style="alloc", hue="alloc", markers=True,
                style_order=order, hue_order=order)
g.set(xticks=[1, 16, 32, 48, 64, 80, 96])
g.set(ylim=(0, 1400))
g.legend.set_title("Allocator")
g.set(ylabel="time in ns")
g.set(xlabel="cores")
g.set_titles("{row_name}, {col_name}")
g.savefig("../../presentation/plot/bulk.pdf")
